<a href="https://colab.research.google.com/github/ShabnaIlmi/Data-Science-Group-Project/blob/master/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Installing the required libraries
!pip install openpyxl

# Importing the relevant libraries
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


In [10]:
# Step 1: Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Step 2: Load the Excel file
file_path = '/content/drive/MyDrive/CM2603 Data Science Group Project/Data Regardings/IMPORT STATISTICS - 2023.xlsx'  # Correct path
excel_data = pd.ExcelFile(file_path)

# Inspecting the Sheet Names
print("Sheet Names:", excel_data.sheet_names)

Sheet Names: ['Dataset_1', 'Dataset_2', 'Dataset_3', 'Dataset_4', 'Dataset_5', 'Dataset_6', 'Dataset_7', 'Dataset_8', 'Dataset_9', 'Dataset_10', 'Dataset_11', 'Dataset_12', 'Dataset_13', 'Dataset_14', 'Dataset_15', 'Dataset_16', 'Dataset_17', 'Dataset_18', 'Dataset_19', 'Dataset_20', 'Dataset_21', 'Dataset_22', 'Dataset_23', 'Dataset_24', 'Dataset_25', 'Dataset_26', 'Dataset_27', 'Dataset_28', 'Dataset_29', 'Dataset_30', 'Dataset_31', 'Dataset_32', 'Dataset_33', 'Dataset_34', 'Dataset_35', 'Dataset_36', 'Dataset_37', 'Dataset_38', 'Dataset_39', 'Dataset_40', 'Dataset_41', 'Dataset_42', 'Dataset_43', 'Dataset_44', 'Dataset_45', 'Dataset_46', 'Dataset_47', 'Dataset_48', 'Dataset_49', 'Dataset_50', 'Dataset_51', 'Dataset_52', 'Dataset_53', 'Dataset_54', 'Dataset_55', 'Dataset_56', 'Dataset_57', 'Dataset_58', 'Dataset_59', 'Dataset_60', 'Dataset_61', 'Dataset_62', 'Dataset_63', 'Dataset_64', 'Dataset_65', 'Dataset_66', 'Dataset_67', 'Dataset_68', 'Dataset_69', 'Dataset_70', 'Dataset_71', '

In [11]:
# Step 3: Loading Data from All Sheets (Process only the first sheet as an example)
data = excel_data.parse(excel_data.sheet_names[0])  # First sheet
print(f"\nLoaded Sheet: {excel_data.sheet_names[0]}")

# Inspecting Data
print("\nFirst 5 rows of the dataset:")
print(data.head())
print("\nDataset Info:")
data.info()
print("\nMissing Values Summary:")
print(data.isnull().sum())


Loaded Sheet: Dataset_1

First 5 rows of the dataset:
                    IMPORTER  YEAR      MONTH  HSCODE      COUNTRY UNIT  \
0       TransWorld Chemicals  2023       JUNE  280110       FRANCE  Kg.   
1  Horizon Industrial Supply  2023  SEPTEMBER  283719         U.K.  Kg.   
2     Greenfield Enterprises  2023       JULY  284700  SOUTH KOREA  Kg.   
3       Redwood Chemical Co.  2023   DECEMBER  280110      AUSTRIA  Kg.   
4       Redwood Chemical Co.  2023      APRIL  280800        SPAIN  Kg.   

   QUANTITY  VALUE_RS                                     DESCRIPTION_01  \
0    179640  13387839  Inorganic chemicals; Organic or inorganic comp...   
1     53784  36871839  Inorganic chemicals; Organic or inorganic comp...   
2     77028  35228981  Inorganic chemicals; Organic or inorganic comp...   
3    137434  23393445  Inorganic chemicals; Organic or inorganic comp...   
4    116881  11490994  Inorganic chemicals; Organic or inorganic comp...   

                                     

In [12]:
# Step 4: Handle Missing Values
# Filling missing numeric columns with the mean, and categorical columns with 'Unknown'
for col in data.columns:
    if data[col].dtype == 'object':
        data[col].fillna('Unknown', inplace=True)
    else:
        data[col].fillna(data[col].mean(), inplace=True)

print("\nAfter Handling Missing Values:")
print(data.isnull().sum())


After Handling Missing Values:
IMPORTER          0
YEAR              0
MONTH             0
HSCODE            0
COUNTRY           0
UNIT              0
QUANTITY          0
VALUE_RS          0
DESCRIPTION_01    0
DESCRIPTION_02    0
DESCRIPTION_03    0
dtype: int64


<ipython-input-12-4428f53fe442>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna('Unknown', inplace=True)
<ipython-input-12-4428f53fe442>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.m

In [13]:
# Step 5: Standardize Column Names
data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_')
print("\nColumns After Standardization:")
print(data.columns)


Columns After Standardization:
Index(['importer', 'year', 'month', 'hscode', 'country', 'unit', 'quantity',
       'value_rs', 'description_01', 'description_02', 'description_03'],
      dtype='object')


In [14]:
# Step 6: Remove Duplicates
data.drop_duplicates(inplace=True)
print("\nAfter Removing Duplicates:")
print(f"Shape: {data.shape}")


After Removing Duplicates:
Shape: (100, 11)


In [15]:
# Step 7: Handle Outliers (Cap numeric columns at the 1st and 99th percentiles)
numeric_cols = data.select_dtypes(include=['number']).columns
for col in numeric_cols:
    lower_bound = data[col].quantile(0.01)
    upper_bound = data[col].quantile(0.99)
    data[col] = data[col].clip(lower=lower_bound, upper=upper_bound)

print("\nAfter Handling Outliers:")
print(data[numeric_cols].describe())


After Handling Outliers:
         year         hscode       quantity      value_rs
count   100.0     100.000000     100.000000  1.000000e+02
mean   2023.0  282323.510000   97047.523900  2.604947e+07
std       0.0    1879.302366   58263.323467  1.401270e+07
min    2023.0  280110.000000    4837.200000  8.980640e+05
25%    2023.0  280110.000000   45205.750000  1.499321e+07
50%    2023.0  283711.000000   90109.500000  2.549791e+07
75%    2023.0  283719.000000  145189.000000  3.816808e+07
max    2023.0  284700.000000  195717.190000  4.932748e+07


In [16]:
# Step 8: Encode Categorical Variables
# One-hot encode categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

print("\nAfter Encoding Categorical Variables:")
print(data.head())


After Encoding Categorical Variables:
   year  hscode  quantity    value_rs  importer_EcoTrade Ltd.  \
0  2023  280110  179640.0  13387839.0                   False   
1  2023  283719   53784.0  36871839.0                   False   
2  2023  284700   77028.0  35228981.0                   False   
3  2023  280110  137434.0  23393445.0                   False   
4  2023  280800  116881.0  11490994.0                   False   

   importer_GlobalTech Imports  importer_Greenfield Enterprises  \
0                        False                            False   
1                        False                            False   
2                        False                             True   
3                        False                            False   
4                        False                            False   

   importer_Horizon Industrial Supply  importer_IronGate Imports Ltd.  \
0                               False                           False   
1                    